In [1]:
# !pip install langchain_groq langchain_core langchain_community

In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "API-KEY",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("Who is lord Ram?")
print(result.content)

Lord Rama is a major deity in Hinduism and is considered the seventh avatar (incarnation) of Lord Vishnu, one of the three primary aspects of the Supreme Being in Hinduism. He is the main character in the ancient Indian epic, the Ramayana, which is attributed to the sage Valmiki.

According to Hindu mythology, Lord Rama was born as a prince in the kingdom of Ayodhya, to King Dasharatha and Queen Kausalya. He is known for his exceptional virtues, such as truthfulness, compassion, and bravery. Rama is often depicted with his wife, Sita, and his brother, Lakshmana.

The story of Lord Rama is as follows:

* Rama was exiled to the forest for 14 years by his father, King Dasharatha, due to a promise made to his stepmother, Queen Kaikeyi.
* During his exile, Rama's wife, Sita, was abducted by the demon king Ravana, who took her to his kingdom in Lanka (modern-day Sri Lanka).
* Rama, with the help of his brother Lakshmana and the monkey god Hanuman, launched a war against Ravana and eventually

In [3]:
# !pip install pypdf

In [4]:
# !pip install sentence_transformers

In [5]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [6]:
def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "API-KEY",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

def create_vector_db():
    loader=DirectoryLoader("data/",glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    text=text_splitter.split_documents(documents)
    embeddings= HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(text, embeddings, persist_directory = './chroma_db')
    vector_db.persist()
    
    print("ChromaDB created and data saved")
    
    return vector_db

def setup_qa_chain(vector_db, llm):
    retriever=vector_db.as_retriever()
    prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """

    PROMPT=PromptTemplate(template=prompt_templates,input_variables=['context','question'])

    qa_chain = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type = "stuff",
        retriever = retriever,
        chain_type_kwargs = {"prompt": PROMPT}
    )
    return qa_chain

def main():
    print("Intializing Chatbot.........")
    llm=initialize_llm()
    db_path="/chroma_db"

    if not os.path.exists(db_path):
        vector_db  = create_vector_db()
    else:
        embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
        vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
    qa_chain = setup_qa_chain(vector_db, llm)

    while True:
        query = input("\nHuman: ")
        if query.lower()  == "exit":
            print("Chatbot: Take Care of yourself, Goodbye!")
            break
        response = qa_chain.run(query)
        print(f"Chatbot: {response}")

if __name__ == "__main__":
  main()

Intializing Chatbot.........


C:\Users\Omkar\AppData\Local\Temp\ipykernel_2920\4045169836.py:14: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings= HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
C:\Users\Omkar\AppData\Local\Temp\ipykernel_2920\4045169836.py:16: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


ChromaDB created and data saved



Human:  hello


C:\Users\Omkar\AppData\Local\Temp\ipykernel_2920\4045169836.py:56: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


Chatbot: Hello! It seems like you've shared a passage about the definition of positive mental health, and how it's not just about individual factors, but also about the social and physical environments that influence us. That's a really important point, as our surroundings and relationships can have a significant impact on our well-being.

I'd love to explore this topic further with you. What are your thoughts on this definition of positive mental health? Do you feel like it resonates with you, or are there other factors that you think are important to consider? Perhaps you've had experiences where your environment or relationships have affected your mental health? I'm here to listen and offer support. How can I assist you today?



Human:  i am mad


Chatbot: I can sense that you're feeling upset and angry right now. It takes a lot of courage to acknowledge and express your emotions, so thank you for sharing that with me. Would you like to talk about what's making you feel mad? Sometimes sharing your thoughts and feelings can help you process and understand them better. I'm here to listen and offer support without judgment. What's been going on that's got you feeling this way?



Human:  exit


Chatbot: Take Care of yourself, Goodbye!


In [19]:
# !pip install gradio

   ---------------------------------------- 0.0/57.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/57.6 MB ? eta -:--:--
   ---------------------------------------- 0.5/57.6 MB 1.2 MB/s eta 0:00:48
    --------------------------------------- 0.8/57.6 MB 1.3 MB/s eta 0:00:44
    --------------------------------------- 1.0/57.6 MB 1.2 MB/s eta 0:00:47
    --------------------------------------- 1.3/57.6 MB 1.3 MB/s eta 0:00:44
   - -------------------------------------- 1.6/57.6 MB 1.3 MB/s eta 0:00:45
   - -------------------------------------- 1.8/57.6 MB 1.3 MB/s eta 0:00:45
   - -------------------------------------- 2.1/57.6 MB 1.3 MB/s eta 0:00:44
   - -------------------------------------- 2.4/57.6 MB 1.3 MB/s eta 0:00:44
   - -------------------------------------- 2.6/57.6 MB 1.3 MB/s eta 0:00:44
   -- ------------------------------------- 2.9/57.6 MB 1.3 MB/s eta 0:00:44
   -- ------------------------------------- 3.1/57.6 MB 1.3 MB/s eta 0:00:44
   -- -------